# Lab 02: A little bit lower API

Trong bài thực hành này:
- Tự thiết kế layer trong Keras
    - Kiểm soát các công thức toán học, những tham số của layer

## Import thư viện và đọc data

In [ ]:
#import thư viện cần thiết
## thư viện machine learning và hỗ trợ
import tensorflow as tf
from tensorflow import keras
import numpy as np

## thư viện để vẽ đồ thị
import matplotlib.pyplot as plt


In [ ]:
# Tải dataset MNIST từ tensorflow
## MNIST là bài toán dự đoán một ảnh thể hiện ký tự số nào

## tải MNIST dataset từ keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = X_train/255.0, X_test/255.0

##in dataset
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
print("Có 60000 ảnh dùng để train và valid, 10000 ảnh dùng để test")
print("Mỗi ảnh có một kênh màu, kích thước 28x28")
print()

## in thử ảnh một ảnh
print("Ảnh đầu tiên của tập train")
print("Label đầu tiên của tập train: ", y_train[0])
plt.imshow(X_train[0], cmap='gray')
plt.show()

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

## Custom Layer Keras


Để tùy chỉnh một layer của Keras cách tốt nhất là viết một lớp con của lớp keras.Layers.Layer trong đó cần override những hàm:
- \_\_init__: khởi tạo, lưu tham số cho layer
- build: thiết lập layer dựa vào shape của input, khai báo các biến, các layer ở hàm này
- call: cài đặt những tính toán feedforward

### Custom Fully Connected layer

In [ ]:
## Viết một fully connected layer dưới dạng custom layer

## import thư viện backend, hỗ trợ nhũng hàm của keras
from keras import backend as K

class FullyConnectedLayer(keras.layers.Layer):
    def __init__(self, n_units=10, activation=tf.nn.sigmoid):
        print("__init__ called")
        ## n_units, activation là tham số do mình tự định nghĩa
        ## n_units thể hiện số nơ-ron của lớp fully connected
        ## activation là hàm kích hoạt
        
        # gọi hàm khởi tạo của lớp cha
        super(FullyConnectedLayer, self).__init__()
        # lưu lại số nơ-ron của lớp này
        self.n_units = n_units
        self.activation_function = activation

    def build(self, input_shape):
        print("build called")
        ## input_shape: là shape của input của layer này
        ## tham số này là bắt buộc
        
        ### Tạo biến có kích thước [input_shape[-1], self.n_units],
        ### các giá trị biến tạo ngẫu nhiên theo phân phối chuẩn mean = 0, std = 0.01
        
        self.kernel = self.add_variable(name='kernel',
                                        shape=[int(input_shape[-1]), self.n_units],
                                        initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.01))
        
        ## Tương tự, khai báo một vector bias
        self.bias = self.add_variable(name='bias',
                                      shape=[self.n_units],
                                      initializer=keras.initializers.Ones())

        
    def call(self, inputs):
        print("call called")

        ## input chính là một lớp keras (hoặc tensor) truyền vào layer này
        ## tham số input là bắt buộc
        
        ## Công thức toán học của lớp này
        ## tensorflow có hỗ broadcasting nên self.bias sẽ được cộng vào từng dòng của ma trận
        matmul = tf.matmul(inputs, self.kernel) + self.bias
        outputs = self.activation_function(matmul)  ##áp dụng hàm kích hoạt
        return outputs

## Thử xem những tham số và kích thước của layer với input là vector (300,)
inputs = keras.layers.Input(shape=(300,))
conv = FullyConnectedLayer(n_units=10)      ## hàm __init__ được gọi
conv_output = conv(inputs)                  ## hàm build và call được gọi

## In các biến train được của layer
## conv.trainable_variables trả về một list các biến train được của layer
print(conv.trainable_variables)
## In output của layer
print(conv_output.shape)


In [ ]:
class ConvMaxpoolBlock(keras.layers.Layer):
    ## Lớp này sẽ xây dựng lớp convolutional layer và maxpooling
    
    
    def __init__(self, filter_size, n_filters):
        ## filter_size sẽ dùng như là kích thước của filter của lớp convolution
        ## là một list 2 phần tử
        ## n_filters là số filter của lớp convolution
        ## là một số nguyên
        
        # gọi hàm khởi tạo của lớp cha
        super(ConvMaxpoolBlock, self).__init__()
        # lưu lại mấy thông số
        self.filter_size = filter_size
        self.n_filters = n_filters
        
    def build(self, input_shape):
        ## ta giả sử input là một tensor (layer) có kích thước [None, height, weight, n_channels]
        
        ## khởi tạo và khai báo kernel
        self.kernel = self.add_variable(name='kernel',
                                        shape=[self.filter_size[0], self.filter_size[1], int(input_shape[-1]), self.n_filters],
                                        initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05))
        
        ## khởi tạo và khai báo bias
        self.bias = self.add_variable(name='bias',
                                      shape=[self.n_filters],
                                        initializer=keras.initializers.Ones())
        
    def call(self, inputs):
        ## phép toán tích chập
        conv = K.conv2d(x=inputs,
                            kernel=self.kernel,
                       padding='same')
        ## cộng bias (có broadcasting hỗ trợ nên lập trình đơn giản)
        conv = conv + self.bias
        ## áp dụng hàm kích hoạt
        conv = tf.nn.relu(conv)
        ## áp dụng maxpooling
        maxpool = K.pool2d(conv, 
                           pool_size=(2,2), 
                           strides=(2,2),
                          pool_mode='max')
        return maxpool

## Thử xem những tham số và kích thước của layer với input là ảnh (300,300,3)
inputs = keras.layers.Input(shape=(300,300,3))
conv = ConvMaxpoolBlock(filter_size=[5,5], n_filters=10)  ##hàm __init__ được gọi
conv_output = conv(inputs)                                ##hàm build và call được gọi

## In các biến train được của layer
print(conv.trainable_variables)
## In output của layer
print(conv_output.shape)

## Bài tập
1. Hãy xây dựng custom layer 

```python
class DoubleFullyConnectedLayer(keras.layers.Layer):
    
    def __init__(self, n_units_1, n_units_2, activation):
```

gồm:
    - 2 lớp Dense chồng nhau: lớp đầu có số neuron là n_units_1, lớp còn lại có số neuron là n_units_2
    - Tự khai báo các biến và thiết lập các phép toán như lớp FullyConnectedLayer ở trên
2. Với lớp được định nghĩa, vãy xây dựng và huấn luyện một model có cấu trúc như sau:
    - Đoán xem trong lớp DoubleFullyConnectedLayer tham số n_units_1 và n_units_2 bằng bao nhiêu?
    
```
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv_maxpool_block_1 (ConvMa (None, 14, 14, 20)        520       
_________________________________________________________________
conv_maxpool_block_2 (ConvMa (None, 7, 7, 20)          10020     
_________________________________________________________________
flatten_1 (Flatten)          (None, 980)               0         
_________________________________________________________________
double_fully_connected_layer (None, 10)                99110     
=================================================================
Total params: 109,650
Trainable params: 109,650
Non-trainable params: 0
_________________________________________________________________
```
